In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Competition/")


In [14]:
with open('Sample3/train_sample3.csv', 'r') as f:
    lst = f.readlines()
print(len(lst))

39973


In [15]:
import json
import sys
sys.path.append("/content/drive/My Drive/Competition/")
from dataset import Drive360Loader

# load the config.json file that specifies data 
# location parameters and other hyperparameters 
# required.
config = json.load(open('./config.json'))

# create a train, validation and test data loader
train_loader = Drive360Loader(config, 'train')
validation_loader = Drive360Loader(config, 'validation')
test_loader = Drive360Loader(config, 'test')

# print the data (keys) available for use. See full 
# description of each data type in the documents.
print('Loaded train loader with the following data available as a dict.')
print(train_loader.drive360.dataframe.keys())

Phase: train # of data: 38328
Phase: validation # of data: 2522
Phase: test # of data: 0
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'canSteering',
       'canSpeed', 'chapter'],
      dtype='object')


In [16]:
config['data_loader']['historic']

{'frequency': 1, 'number': 3}

In [0]:
import pickle
import gzip

def preprocess_data(loader, size_limit, name):
  
  # dump to pickle file by batch
  with gzip.open(f'{name}.pickle.gz', 'wb') as gz:
    
      # pre-process trading data and deploy it to GPU
      processed_data = []  # list of tuple (data, target)

      for batch_idx, (data, target) in enumerate(loader):
          for k, v in data.items():
              if 'Front' in k:
                for kk, vv in v.items():
                    data[k][kk] = vv.cuda()
              else:
                data[k] = v.cuda()

          for k, v in target.items():
              target[k] = v.cuda()

          processed_data.append((data, target))

          if batch_idx > 0 and batch_idx % 20 == 0:
            print(f'{name}: {batch_idx}')
            pickle.dump(processed_data, gz, pickle.HIGHEST_PROTOCOL)
            processed_data = []

          if size_limit > 0 and batch_idx >= size_limit - 1:
              break

      if processed_data:
            print(f'{name}: {batch_idx}')
            pickle.dump(processed_data, gz, pickle.HIGHEST_PROTOCOL)
            processed_data = []

In [0]:
processed_train_data = preprocess_data(train_loader, -1, 'processed_train_data_full')
#processed_validation_data = preprocess_data(validation_loader, -1, 'processed_validation_data_full')
#processed_test_data = preprocess_data(test_loader, -1, 'processed_test_data_full')

processed_train_data_full: 20
